In [1]:
import pandas as pd
poetry_df=pd.read_csv('poetry.csv')
author_df=pd.read_csv('author.csv')

In [2]:
poetry_df.set_index('id')
poetry_df.sample(10)

,Unnamed: 0,id,title,yunlv_rule,author_id,content,dynasty,author
227118,227118,227119,送范貫之,仄仄平平仄，平平仄仄平。|平平仄平仄，仄仄仄平平。|仄仄平平仄，平平仄仄平。|平平仄平仄，平...,4935,范叔貧如此，誰當與敝袍。|哀歌困牛角，異味笑鄉豪。|相得空長鋏，徒歸欲二毛。|多慚訪夷節，無...,S,劉敞
177705,177705,177706,扶杖而東渡五位橋曲折而北松下逢道人賢公喜爲之詩曰,平仄平○仄仄，平平平仄平○。|平仄仄平○仄，仄平仄仄平平。,6539,賢也嶔嵌歷落，軒然頤頬開張。|松下偶然相值，立談愛子清狂。,S,釋德洪
52704,52704,52705,謁墓五首 其四,平仄平平平仄仄，平平仄仄平平平。|平平仄仄平平仄，仄仄平平仄仄平。,9758,庵裏僧應行脚去，龕中佛也爲眉嚬。|炎炎昔有□□者，寂寂今無上冢人。,S,劉克莊
128303,128303,128304,泊福山港,仄平仄仄平仄平，仄○仄仄平平平。|平平仄仄仄仄仄，仄○平仄平平平。|平平平仄仄平仄，平平仄仄...,8099,我舟欲渡風伯嗔，卷帆夜泊滄江津。|江聲拍枕冷不寐，起看河漢無纖塵。|須臾風定水波伏，江妃湧起...,S,崔敦禮
74921,74921,74922,落花,仄仄平平仄仄平，平平仄仄仄平平。|平平仄仄平平仄，仄仄平平仄仄平。,9212,拾得紅英不忍殘，殷勤更覓舊枝安。|情知不似開時好，猶勝溪頭水面看。,S,李檝
6744,6744,6745,呈蕭大帥,仄仄平平仄仄平，平平平仄仄平平。|仄平平仄平平仄，平仄平平仄仄平。|仄仄仄平平仄仄，平平仄仄...,11121,太掖勾陳瑞靄浮，宮花時綴五雲裘。|湛盧光截飛狐月，繁弱風號涿鹿秋。|蟣蝨幾曾生介胄，貂蟬又見...,S,黄宏
125669,125669,125670,和范待制秋興三首 其三,平仄平平仄仄平，仄平仄仄仄平平。|仄仄仄仄平平仄，仄仄平平仄仄平。|仄仄仄平平仄仄，仄平平仄...,8211,山澤沉冥氣尚豪，鬢絲未遽嘆蕭騷。|已忘海運鯤鵬化，那計風微燕雀高。|萬里客魂迷楚峽，五更歸夢...,S,陸游
100410,100410,100411,題文與可山水,仄仄平平仄，平平仄仄平。|仄平平仄仄，平仄仄平平。,8424,疎篠寒林手，文家富襪材。|片縑雲氣足，彷彿認蓬萊。,S,許及之
62701,62701,62702,次韻虞退夫除夕七絕句 其四,平仄平平仄仄平，平平仄仄仄平平。|仄平平仄仄平仄，仄仄平平仄仄平。,9536,雲外銜山日半環，傳盃共惜片雲懽。|莫疑來日是來歲，萬古光陰只麽看。,S,魏了翁
231873,231873,231874,題新定紅渠閣,平仄平平仄仄平，仄平平仄仄平平。|仄平平仄平平仄，仄仄平平仄仄平。|仄仄平平平仄仄，平平平仄...,4749,煙卷龍綃豔格殊，彩霞翻處好風俱。|幾杯仙露含秋色，一曲蟾宮在畫圖。|雨送清香浮酒面，泉飛幽韻...,S,張伯玉


In [3]:
poetry_df.dtypes

Unnamed: 0     int64
id             int64
title         object
yunlv_rule    object
author_id      int64
content       object
dynasty       object
author        object
dtype: object

In [4]:
poetry_df['content'] = poetry_df['content'].fillna("")
poetry_df['title'] = poetry_df['title'].fillna("")
poetry_df['author'] = poetry_df['author'].fillna("")

In [5]:
from hanziconv import HanziConv as HC
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 假设 HC.toSimplified 是转换函数
def process_row(row):
    row['content'] = HC.toSimplified(row['content'])
    row['title'] = HC.toSimplified(row['title'])
    row['author'] = HC.toSimplified(row['author'])
    return row

# 并行处理并显示进度
def parallel_process_with_progress(df):
    results = []
    with ThreadPoolExecutor(14) as executor:
        # 创建任务
        tasks = {executor.submit(process_row, row): i for i, row in df.iterrows()}
        # 使用 tqdm 显示进度
        for future in tqdm(as_completed(tasks), total=len(tasks)):
            results.append(future.result())
    return pd.DataFrame(results)

# 转换
poetry_df = parallel_process_with_progress(poetry_df)

100%|██████████| 311828/311828 [00:01<00:00, 218975.97it/s]


In [6]:
poetry_df.sample(10)

,Unnamed: 0,id,title,yunlv_rule,author_id,content,dynasty,author
151229,151229,151230,喜雪,平平仄仄仄仄仄，仄平○仄平仄平。|仄平平仄仄仄仄，仄平○仄仄仄平。,7326,皇家有道岁不恶，雪花如絮千里飞。|老农夸杀麦本绿，一杯汤饼令汝肥。,S,李若水
252034,252034,252035,句 其二一,平○平平仄仄仄，平仄平平仄仄○。,3970,心如莲花不着水，身似翔鸿不可笼。,S,晁逈
119934,119934,119935,述志二首 其二,平平仄仄仄平平，平仄平平仄仄平。|平仄仄平平仄仄，平平平仄仄平平。,8211,新丰买酒慰无聊，冲雪骑驴上灞桥。|罴老尚堪吞貈子，松风何至羡山苗。,S,陆游
170618,170618,170619,灯碑五首 其四,仄仄平平仄仄平，平平仄仄仄平平。|平平仄仄平平仄，平仄平平仄仄平。,6776,九夏南风入舜琴，恩风泽雨浃飞沈。|陪京最是仪形地，先识君王解愠心。,S,宇文虚中
35727,35727,35728,海门,平平仄仄平，仄仄仄平平。|平仄平平仄，平平仄仄平。|平平仄仄仄，仄仄仄平平。|仄仄平平仄，平...,10236,微光动海门，色界一层新。|玄雾横金阙，丹英吐玉津。|长疑浴日处，上有饮霞人。|顷刻晨烟合，纷...,S,陈淳祖
52229,52229,52230,赠钱相士,仄仄平平仄仄平，仄平平仄仄平平。|仄平仄仄平平仄，平仄平平仄仄平。,9793,我貌君容一种寒，鹭漂鸥泊廿来年。|我诗吟就无人买，君相公卿煞得钱。,S,许棐
43869,43869,43870,六祖赞,仄仄平平仄仄平，仄平仄仄仄平平。|平平仄仄仄仄仄，仄仄平平仄仄平。,9999,不作樵夫作碓夫，只将脚力验精粗。|知佗踏着踏不着，和米和糠到钵盂。,S,释了惠
144575,144575,144576,櫂歌,仄平仄仄平平仄，仄平仄仄平平仄。|平平平仄仄平平，平平仄仄平平仄。|○平仄仄仄仄○，平平仄仄...,7499,谪仙醉揖金陵月，浩歌目送飞鸿绝。|江城樽俎照清辉，楼台忽作烟霞宅。|长杯引竭北斗空，吴歈楚舞...,S,曹勋
21848,21848,21849,写心五首 其二,平仄仄仄平，仄平仄平仄。|平仄平平平，仄平平仄○。|平平仄仄仄，仄仄仄仄仄。|仄仄平○平，○...,10579,今日忽不愁，脱然体无病。|餬饰窗櫺明，整齐书册正。|生无快意事，适此悦我性。|雨足明当晴，更...,S,方回
84320,84320,84321,甘园,平仄平平仄仄平，平平仄仄仄平平。|仄平平仄仄平仄，仄仄平平仄仄平。,8848,曾入当年富贵家，东风几度换年华。|满园桃李不知主，独自开花自落花。,S,吕炎


In [7]:
df_TS_all=poetry_df
print(len(df_TS_all))
df_TS=df_TS_all

311828


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel,TextGenerationPipeline
# 本地模型路径
local_model_path = './gpt2-chn'
tokenizer = BertTokenizer.from_pretrained(local_model_path)
model = GPT2LMHeadModel.from_pretrained(local_model_path)


In [10]:
poem_begin = '[CLS]'  # 诗句开始的标志
poem_sep = '[SEP]'    # 诗句结束的标志
title_separator = ' ： '  # 标题与诗句之间的分隔符
disallow = ['（', '）', '(', ')', '__', '《', '》', '【', '】','+','-','/']
df_TS['fullpoem']=''
df_TS.dropna()
poems=[]
poems_tc=pd.DataFrame(columns=['title','content'])
titles=[]
contents=[]
for i in range(0,len(df_TS)):  # 遍历指定列，跳过缺失值
    ignore_flag =False
    title=df_TS['title'].iloc[i]
    content=df_TS['content'].iloc[i]
    for dis_word in disallow:
        if dis_word in content or dis_word in content:
            ignore_flag = True
            break
    
    for line in content.split('|'):
        line = line.strip()
        if len(line)!=16 :
            ignore_flag = True
            break
    if ignore_flag:
        continue
    content1 = content.replace('|', '') 
    content_with_space = ' '.join(content1)
    title_with_space=' '.join(title) 
    poem=poem_begin+title_with_space+title_separator+content_with_space+poem_sep
    content2=poem_begin+content_with_space+poem_sep
    poems.append(poem)
    titles.append(title_with_space)
    contents.append(content2)

In [11]:
print(len(poems))
df_poems = pd.DataFrame(poems, columns=['fullpoem'])



166016


In [12]:
poems_tc['title']=titles
poems_tc['content']=contents
poems_tc.head(10)

,title,content
0,送 乘 侍 者 归 宁,[CLS]佛 既 强 名 嫌 不 做 ， 亲 犹 未 老 已 归 休 。 信 邪 倒 见 难...
1,和 汪 正 夫 梅 其 三,[CLS]江 南 江 北 谪 仙 才 ， 爲 爱 湖 山 特 地 来 。 愿 得 春 寒 过...
2,孙 侯 见 和 复 次 韵 五 首 其 二,[CLS]我 亦 闻 絃 知 雅 曲 ， 松 风 声 不 类 鸣 鸾 。 相 逢 意 气 须...
3,寄 焦 山 成 老 高 邮 滋 老,[CLS]他 乡 卧 病 白 头 翁 ， 孤 愤 犹 存 涕 泪 中 。 竹 叶 有 阴 唯...
4,送 希 古,[CLS]清 都 不 敢 梦 中 游 ， 何 况 通 明 侍 冕 旒 。 分 手 知 从 天...
5,爱 闲,[CLS]爱 闲 惟 与 病 相 宜 ， 壮 岁 怀 归 老 可 知 。 睡 熟 素 书 横...
6,宜 兴 道 中,[CLS]客 怀 无 事 只 增 愁 ， 遮 眼 残 书 懒 即 休 。 午 醉 忽 惊 如...
7,次 韵 寄 谢 杨 制 机,[CLS]把 卷 怆 然 不 自 胜 ， 古 来 离 合 岂 人 能 。 未 应 鬻 髢 尤...
8,出 南 郊 清 甚,[CLS]春 风 吹 屣 到 南 园 ， 着 酒 涂 酥 花 正 繁 。 更 喜 满 前 无...
9,和 汪 正 夫 梅 其 二,[CLS]世 上 庸 儿 苦 忌 才 ， 脱 身 曾 把 虎 须 来 。 他 年 若 隠 吴...


In [13]:
df_poems.sample(10)

,fullpoem
159039,[CLS]次 天 启 赠 浄 名 吴 道 人 韵 ： 君 名 讵 要 书 黄 纸 ， 儿 曹...
69822,[CLS]次 韵 丁 端 叔 舟 中 值 雪 三 绝 其 三 ： 延 和 初 奏 三...
67104,[CLS]王 子 可 欲 寿 赵 寅 仲 思 成 出 薲 嚚 闽 兟 瞤 五 字 令 赋 诗...
63521,[CLS]飞 玉 亭 ： 素 练 飞 来 衮 雪 毬 ， 满 怀 风 露 不 胜 秋 。 翠...
131689,[CLS]游 古 陌 吟 ： 谁 言 今 古 事 难 穷 ， 大 抵 荣 枯 总 是 空 。...
114831,[CLS]初 赐 幡 胜 戏 和 诸 公 二 首 其 二 ： 拜 赐 匆 匆 早 上...
132920,[CLS]送 李 伯 和 吏 部 提 举 浙 东 伯 珍 寺 丞 将 漕 夔 门 ： 奕 奕...
69456,[CLS]春 雨 戏 赵 立 之 ： 花 枝 欲 动 濡 须 坞 ， 无 頼 春 风 吹 小...
56938,[CLS]黄 雀 食 新 二 首 其 一 ： 鹅 黄 染 线 织 秋 衣 ， 杨 柳...
161003,[CLS]髀 肉 生 ： 归 计 无 成 须 绝 望 ， 幽 居 有 意 漫 经 营 。 自...


In [14]:
from datasets import Dataset
max_length=128
def tokenize_function(examples):
    encodings = tokenizer(examples['fullpoem'], padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    # 设置标签为右移一个字符的结果
    input_ids = encodings['input_ids']
    labels = input_ids.clone()
    encodings['labels'] = labels
    return encodings

# 使用 datasets 库的 Dataset 对象来加载数据
dataset = Dataset.from_pandas(df_poems)

# 应用标记化
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 检查处理后的数据
tokenized_dataset[0]

tokenized_dataset.set_format(type='torch', columns=['labels','input_ids', 'attention_mask'])

Map:   0%|          | 0/166016 [00:00<?, ? examples/s]

In [15]:
from transformers import Trainer, TrainingArguments,EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',             # 训练结果保存目录
    num_train_epochs=20,                 # 训练轮次
    per_device_train_batch_size=128,  # 相对较小的批次大小
    gradient_accumulation_steps=4,   # 累积梯度以模拟更大的批次  
    save_steps=200,                     # 每200步保存一次模型
    save_total_limit=2,                 # 保留最新的2个模型
    logging_dir='./logs',               # 日志目录
    logging_steps=100,                   # 每100步记录一次日志
    fp16=True, 
)



In [16]:
from transformers import GPT2LMHeadModel
# 设置训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 加载模型并移动到设备上
model = model.to(device)
# 打印模型所在的设备
print(f'Model is loaded on device: {model.device}')

Model is loaded on device: cuda:0


In [17]:

# 使用 Trainer 进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,

)

trainer.train()

Step,Training Loss
100,4.471600
200,1.434400
300,1.383600
400,1.331800
500,1.319300
600,1.306300
700,1.278200
800,1.275300
900,1.272000
1000,1.249900


TrainOutput(global_step=6480, training_loss=1.2210377763818812, metrics={'train_runtime': 6756.2482, 'train_samples_per_second': 491.444, 'train_steps_per_second': 0.959, 'total_flos': 2.16249466355712e+17, 'train_loss': 1.2210377763818812, 'epoch': 19.940632228218966})

In [20]:
model.save_pretrained('./fine_tuned_gpt2_chn_7-tmp1')
tokenizer.save_pretrained('./fine_tuned_gpt2_chn_7-tmp1')

('./fine_tuned_gpt2_chn_7-tmp1/tokenizer_config.json',
 './fine_tuned_gpt2_chn_7-tmp1/special_tokens_map.json',
 './fine_tuned_gpt2_chn_7-tmp1/vocab.txt',
 './fine_tuned_gpt2_chn_7-tmp1/added_tokens.json')

In [21]:
from transformers import BertTokenizer, GPT2LMHeadModel,TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained('./fine_tuned_gpt2_chn_7-tmp1')
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2_chn_7-tmp1')
text_generator = TextGenerationPipeline(model, tokenizer)   


Device set to use cuda:0


In [22]:
text_generator("[CLS]北 山 雪 其 二", max_length=100, do_sample=True,temperature=0.8)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '[CLS]北 山 雪 其 二 ： 青 山 淡 淡 水 悠 悠 ， 落 日 孤 云 处 处 愁 。 薄 宦 不 归 何 所 乐 ， 沧 浪 一 曲 思 夷 犹 。 □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □'}]

In [26]:
text_generator("[CLS]观 花 赠 杜 甫", max_length=80, do_sample=True,temperature=0.8)

[{'generated_text': '[CLS]观 花 赠 杜 甫 ： 红 紫 纷 纷 竞 赏 时 ， 一 番 风 雨 一 番 诗 。 要 知 桃 李 成 阴 处 ， 须 信 松 篁 有 岁 迟 。'}]

In [24]:
text_generator("[CLS]观 秋 其 八", max_length=100, do_sample=True,temperature=0.9)

[{'generated_text': '[CLS]观 秋 其 八 ： 三 年 病 鹤 伴 残 骸 ， 二 十 重 来 上 翠 崖 。 已 觉 山 中 添 笋 蕨 ， 何 妨 门 外 长 蒿 莱 。 云 深 路 断 樵 僧 返 ， 风 急 林 寒 钓 叟 来 。 安 得 谢 公 诗 一 首 ， 次 山 书 后 始 爲 回 。'}]

In [ ]:
from transformers import BertTokenizer, GPT2LMHeadModel,TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained('./gpt2-chn')
model = GPT2LMHeadModel.from_pretrained('./gpt2-chn')
text_generator = TextGenerationPipeline(model, tokenizer)   
text_generator("[CLS]雪 夜 上 梁 山，", max_length=100, do_sample=True)